# 1.数据导入与划分

In [1]:
from pyspark.sql import SparkSession # SparkSession 是Spark 2.0版本的新入口
spark = SparkSession.builder.master('local').getOrCreate()

In [2]:
raw_data = spark.read.csv(path="hdfs://localhost:9000/user/bdlab/lab2/SUSY.csv.gz",header=False,inferSchema=True)

In [29]:
raw_data.count()

5000000

In [3]:
train,test = raw_data.randomSplit([0.8,0.2])

In [45]:
raw_data.groupBy(raw_data[0]).count().collect()

[Row(_c0=0.0, count=2712173), Row(_c0=1.0, count=2287827)]

0/1 基本概率

In [4]:
prob_0 = 2712173/5000000
prob_1 = 1-prob_0

In [5]:
# 训练集0/1分布
train.groupBy(raw_data[0]).count().collect()

[Row(_c0=0.0, count=2170505), Row(_c0=1.0, count=1829609)]

In [6]:
# 测试集0/1分布
test.groupBy(raw_data[0]).count().collect()

[Row(_c0=0.0, count=541668), Row(_c0=1.0, count=458218)]

In [ ]:
# 划分后写入hdfs
test.write.csv(path="hdfs://localhost:9000/user/bdlab/lab2/SUSY_test.csv.gzip",compression="gzip")
train.write.csv(path="hdfs://localhost:9000/user/bdlab/lab2/SUSY_train.csv.gzip",compression="gzip")

In [3]:
# 加载train
train = spark.read.csv(path="hdfs://localhost:9000/user/bdlab/lab2/SUSY_train.csv.gzip",header=False,inferSchema=True)

In [13]:
train.write.csv(path="hdfs://localhost:9000/user/bdlab/lab2/SUSY_train.csv.gz",compression="gzip")

测试集处理

In [ ]:
test_data = test.select(test.columns[1:])

In [ ]:
test_label = test.select(test.columns[0])

# 2.Naive-Bayes

In [5]:
import pandas as pd
import numpy as np

In [4]:
cal_mean_func = { _:"mean" for _ in train.columns[1:] }
cal_var_func = { _:"variance" for _ in train.columns[1:]}

In [5]:
mean_train = train.groupBy(train[0]).agg(cal_mean_func).collect()
var_train = train.groupBy(train[0]).agg(cal_var_func).collect()

In [9]:
mean_0 = [ mean_train[0]["avg(_c"+str(i)+str(")")] for i in range(1,19)]
mean_1 = [ mean_train[1]["avg(_c"+str(i)+str(")")] for i in range(1,19)]
var_0 = [ var_train[0]["variance(_c"+str(i)+str(")")] for i in range(1,19)]
var_1 = [ var_train[0]["variance(_c"+str(i)+str(")")] for i in range(1,19)]

In [13]:
# 保存统计信息
np.savetxt("nb_sta",[mean_0,mean_1,var_0,var_1])

In [6]:
# 读取统计信息
mean_0,mean_1,var_0,var_1 = np.loadtxt("nb_sta")

In [29]:
const_pi = 1/ np.sqrt(2*np.pi)
def GaussianProb(x,miu,sigmaq):
    left = const_pi / np.sqrt(sigmaq)
    right = np.exp(- (x - miu)**2 / (2 * sigmaq))
    return left*right

def NB_classifier(x):
    """
    input:
        x: a vec with len of 18
    ouput:
        result: 0 or 1
    """
    if len(x) != 18:
        raise ValueError
        
    # 计算 0 概率
    result_0 = prob_0
    for i in range(18):
        result_0 = result_0 * GaussianProb(x[i],mean_0[i],var_0[i])
        
    # 计算 1 概率
    result_1 = prob_1
    for i in range(18):
        result_1 = result_1 * GaussianProb(x[i],mean_1[i],var_1[i])
    return 0 if result_0>result_1 else 1

def NB_predict(testset):
    return [ NB_classifier(_) for _ in testset ]

### 测试

In [6]:
from sklearn.metrics import accuracy_score,recall_score,classification_report

def metric(true,pre):
    print("Accuracy: {:.4f}, Recall: {:.4f} ".format(accuracy_score(true,pre),recall_score(true,pre)))
    print()
    print(classification_report(true,pre))

In [7]:
test_data_arr = np.load("test_data_arr.npy")
test_label_arr = np.load("test_label_arr.npy")

In [30]:
test_pre_nb = NB_predict(test_data_arr)

In [45]:
metric(test_label_arr,test_pre_nb)

Accuracy: 0.7360, Recall: 0.6047 

             precision    recall  f1-score   support

        0.0       0.72      0.85      0.78    541668
        1.0       0.77      0.60      0.68    458218

avg / total       0.74      0.74      0.73    999886



# 3.Logistic-Regression

In [8]:
sample_train = train.sample(0.1)

In [76]:
sample_train_list = sample_train.collect()
np.save("sample_train_list",sample_train_list)

In [9]:
# n 特征数
n = 18
theta = np.zeros(n+1) # 在LR_fit 中更新

def sigmod(x):
    return 1/(1+np.exp( - np.matmul(x,theta)))

# 对数损失函数
def cost_mapper(x):
    h = sigmod(x[0])
    L = 0
    if x[1]==0:
        L = np.log(1-h)
    else:
        L = np.log(h)
    return -L 

# 计算梯度 (h(x)-y)*x
def grad_mapper(x):
    return (sigmod(x[0]) - x[1])*x[0]

def sum_reducer(x,y):
    # (grad,cost)
    return (x[0]+y[0],x[1]+y[1])

In [10]:
import time
def LR_fit(data,max_iter=30,alpha=0.1,tol=0.001,penalty=0.1):
    """
    input:
        data: rdd [x0,x1,...,xn], x0为label
        max_iter : 最大迭代次数
        alpha : 学习率
        tol: 收敛容忍度
        penalty: 惩罚系数
    output:
        theta: 
    """
    iter_cnt = 0
    m = data.count() # 样本数
    # x => ((1,x1,x2,...,xn),y)
    data = data.map(lambda x:(np.asarray((1,)+x[1:]),x[0]) )
    cost_pre = 0
    time_start = time.time()
    while iter_cnt<max_iter:
        grad,cost = data.map(lambda x:(grad_mapper(x),cost_mapper(x))).reduce(sum_reducer)
        # 计算代价
        cost = cost/m + penalty*np.sum(np.square(theta))/(2*m)
        # 更新 Θ
        theta[0] = theta[0] - alpha*grad[0] /m
        theta[1:] = theta[1:] - alpha*(grad[1:]+penalty*theta[1:]) /m
        
        cost_del = cost_pre - cost
        cost_del = -cost_del if cost_pre==0 else cost_del 
        time_end = time.time()
        print("iter {}, cost={:.8f}, △cost={:.8f}, △time={:.2f}".format(iter_cnt,cost,cost_del,time_end-time_start))
        
        if cost_del<tol:
            print("收敛,算法结束")
            return theta
        cost_pre = cost
        time_start = time_end
        iter_cnt +=1
        
    print("迭代上限,算法结束")
    return theta

In [11]:
# 分类器
def LR_classifier(x):
    h = sigmod(np.insert(x,0,1))
    return 0 if h<0.5 else 1

def LR_predict(testset):
    return [ LR_classifier(_) for _ in testset ]

### 模型训练与测试

0.4-0.2-0.2 训练

In [27]:
# stage1 10%data alpha=0.4
theta = LR_fit(data=sample_train.rdd,alpha=0.4,tol=0.002)

KeyboardInterrupt: 

In [21]:
# stage2 10%data alpha=0.2
theta = LR_fit(data=sample_train.rdd,alpha=0.2,tol=0.002)

iter 0, cost=0.58469059, △cost=0.58469059, △time=93.21
iter 1, cost=0.58268623, △cost=0.00200436, △time=92.37
iter 2, cost=0.58074209, △cost=0.00194414, △time=92.09
收敛,算法结束


In [26]:
# stage3 100%data alpha=0.2
theta = LR_fit(data=train.rdd,alpha=0.2,tol=0.001)

iter 0, cost=0.57560294, △cost=0.57560294, △time=166.91
iter 1, cost=0.57388004, △cost=0.00172290, △time=168.13
iter 2, cost=0.57220583, △cost=0.00167421, △time=167.17
iter 3, cost=0.57057770, △cost=0.00162813, △time=167.07
iter 4, cost=0.56899366, △cost=0.00158404, △time=166.21
iter 5, cost=0.56745191, △cost=0.00154175, △time=166.05
iter 6, cost=0.56595076, △cost=0.00150114, △time=168.91
iter 7, cost=0.56448864, △cost=0.00146212, △time=168.95
iter 8, cost=0.56306404, △cost=0.00142461, △time=168.17
iter 9, cost=0.56167552, △cost=0.00138852, △time=167.17
iter 10, cost=0.56032174, △cost=0.00135378, △time=165.93
iter 11, cost=0.55900140, △cost=0.00132034, △time=165.53
iter 12, cost=0.55771329, △cost=0.00128811, △time=164.68
iter 13, cost=0.55645623, △cost=0.00125705, △time=165.27
iter 14, cost=0.55522913, △cost=0.00122710, △time=165.05
iter 15, cost=0.55403093, △cost=0.00119820, △time=165.52
iter 16, cost=0.55286062, △cost=0.00117031, △time=166.15
iter 17, cost=0.55171725, △cost=0.0011433

In [28]:
test_pre_lr = LR_predict(test_data_arr)
metric(test_label_arr,test_pre_lr)

Accuracy: 0.7716, Recall: 0.6175 

             precision    recall  f1-score   support

        0.0       0.74      0.90      0.81    541668
        1.0       0.84      0.62      0.71    458218

avg / total       0.78      0.77      0.77    999886



In [30]:
np.save("theta77166175",theta)

0.1-0.2-0.2 训练

In [11]:
# stage1 10%data alpha=0.1
theta = LR_fit(data=sample_train.rdd)

iter 0, cost=0.69314718, △cost=0.69314718, △time=52.41
iter 1, cost=0.68812175, △cost=0.00502543, △time=52.52
iter 2, cost=0.68412334, △cost=0.00399840, △time=52.47
iter 3, cost=0.68061737, △cost=0.00350597, △time=51.75
iter 4, cost=0.67737519, △cost=0.00324218, △time=52.28
iter 5, cost=0.67429758, △cost=0.00307762, △time=52.35
iter 6, cost=0.67134000, △cost=0.00295758, △time=52.11
iter 7, cost=0.66848116, △cost=0.00285883, △time=52.64
iter 8, cost=0.66570977, △cost=0.00277139, △time=51.93
iter 9, cost=0.66301888, △cost=0.00269089, △time=52.82
iter 10, cost=0.66040354, △cost=0.00261534, △time=53.03
iter 11, cost=0.65785975, △cost=0.00254378, △time=51.44
iter 12, cost=0.65538407, △cost=0.00247569, △time=51.48
iter 13, cost=0.65297334, △cost=0.00241072, △time=51.24
iter 14, cost=0.65062470, △cost=0.00234864, △time=51.40
iter 15, cost=0.64833544, △cost=0.00228926, △time=51.18
iter 16, cost=0.64610305, △cost=0.00223239, △time=53.68
iter 17, cost=0.64392516, △cost=0.00217789, △time=53.89


KeyboardInterrupt: 

In [18]:
# stage2 10%data alpha=0.2
theta = LR_fit(data=sample_train.rdd,alpha=0.2,tol=0.002)

iter 0, cost=0.64179954, △cost=0.64179954, △time=52.82
iter 1, cost=0.63767306, △cost=0.00412649, △time=52.32
iter 2, cost=0.63373512, △cost=0.00393794, △time=52.44
iter 3, cost=0.62997164, △cost=0.00376347, △time=52.82
iter 4, cost=0.62637007, △cost=0.00360157, △time=52.25
iter 5, cost=0.62291915, △cost=0.00345092, △time=52.29
iter 6, cost=0.61960878, △cost=0.00331037, △time=53.00
iter 7, cost=0.61642983, △cost=0.00317895, △time=54.95
iter 8, cost=0.61337403, △cost=0.00305580, △time=52.20
iter 9, cost=0.61043388, △cost=0.00294015, △time=56.33
iter 10, cost=0.60760253, △cost=0.00283135, △time=54.16
iter 11, cost=0.60487370, △cost=0.00272883, △time=54.05
iter 12, cost=0.60224165, △cost=0.00263205, △time=52.29
iter 13, cost=0.59970109, △cost=0.00254056, △time=51.92
iter 14, cost=0.59724713, △cost=0.00245396, △time=52.47
iter 15, cost=0.59487527, △cost=0.00237187, △time=51.94
iter 16, cost=0.59258131, △cost=0.00229396, △time=53.57
iter 17, cost=0.59036137, △cost=0.00221994, △time=52.39
it

KeyboardInterrupt: 

In [23]:
# stage3 100%data alpha=0.2
theta = LR_fit(data=train.rdd,alpha=0.2,tol=0.001)

iter 0, cost=0.55570630, △cost=0.55570630, △time=90.74
iter 1, cost=0.55449077, △cost=0.00121553, △time=95.63
iter 2, cost=0.55330378, △cost=0.00118699, △time=97.15
iter 3, cost=0.55214434, △cost=0.00115944, △time=96.55
iter 4, cost=0.55101150, △cost=0.00113283, △time=100.65
iter 5, cost=0.54990438, △cost=0.00110713, △time=97.53
iter 6, cost=0.54882209, △cost=0.00108228, △time=99.60
iter 7, cost=0.54776383, △cost=0.00105826, △time=96.68
iter 8, cost=0.54672880, △cost=0.00103503, △time=97.20
iter 9, cost=0.54571626, △cost=0.00101254, △time=97.31
iter 10, cost=0.54472548, △cost=0.00099078, △time=98.00
收敛,算法结束


In [27]:
test_pre_lr = LR_predict(test_data_arr)
metric(test_label_arr,test_pre_lr)

Accuracy: 0.7716, Recall: 0.6170 

             precision    recall  f1-score   support

        0.0       0.74      0.90      0.81    541668
        1.0       0.84      0.62      0.71    458218

avg / total       0.78      0.77      0.77    999886



In [29]:
np.save("theta77166170",theta)

np.array 关于向量乘数值的运算速度测试

In [31]:
%%time
def grad_mapper(x):
    return (sigmod(x[0]) - x[1])*x[0]
grad,cost = sample_train.rdd.map(lambda x:(np.asarray((1,)+x[1:]),x[0]) ).map(lambda x:(grad_mapper(x),cost_mapper(x))).reduce(sum_reducer)

CPU times: user 15.5 ms, sys: 16.6 ms, total: 32.2 ms
Wall time: 52.9 s


In [32]:
%%time
def grad_mapper(x):
    h = (sigmod(x[0]) - x[1])
    return [ h*_ for _ in x[0]]
grad,cost = sample_train.rdd.map(lambda x:(np.asarray((1,)+x[1:]),x[0]) ).map(lambda x:(grad_mapper(x),cost_mapper(x))).reduce(sum_reducer)

CPU times: user 231 ms, sys: 78.7 ms, total: 309 ms
Wall time: 3min 3s


In [33]:
%%time
def grad_mapper(x):
    h = (sigmod(x[0]) - x[1])
    return [ h*_ for _ in x[0]]
grad,cost = sample_train.rdd.map(lambda x:((1,)+x[1:],x[0]) ).map(lambda x:(grad_mapper(x),cost_mapper(x))).reduce(sum_reducer)

CPU times: user 204 ms, sys: 41.4 ms, total: 246 ms
Wall time: 2min 50s


In [25]:
len(sample_train_list)

40230